## Graph-level inputs and outputs in a WorkGraph

In `WorkGraph`, you can define **graph-level inputs and outputs** to:

* **Simplify the interface** by exposing only high-level parameters to users.
* **Share inputs** across multiple tasks in the graph.
* **Aggregate outputs** from internal tasks, optionally organizing or renaming them.

This allows you to build clean, modular, and reusable workflows.


### Example: Using Graph-Level Inputs and Outputs



In [1]:
from aiida_workgraph import WorkGraph, task
from aiida import load_profile

load_profile()

@task
def add(x, y):
    """Simple addition task."""
    return x + y

# Create a new WorkGraph
wg = WorkGraph("test_graph_inputs_outputs")

# Define graph-level input
wg.add_input("workgraph.any", "a")

# Add tasks using the graph-level input
wg.add_task(add, "add1", x=wg.inputs.a, y=3)
wg.add_task(add, "add2", x=wg.inputs.a, y=wg.tasks.add1.outputs.result)

# Define graph-level outputs by exposing selected task results
wg.outputs.sum1 = wg.tasks.add1.outputs.result
wg.outputs.sum2 = wg.tasks.add2.outputs.result
wg.to_html()

# Run the workgraph


In [2]:
# set the graph-level input
wg.inputs.a = 1
wg.submit(wait=True)

# Print the graph-level outputs
print("Graph-level outputs:")
print("sum1:", wg.outputs.sum1.value)
print("sum2:", wg.outputs.sum2.value)

WorkGraph process created, PK: 53194
Process 53194 finished with state: FINISHED
Graph-level outputs:
sum1: uuid: 28fa0b1f-37b3-4796-9281-d64d15276982 (pk: 53199) value: 4
sum2: uuid: b90c7ff5-4945-4b55-9e69-f76b1b71a360 (pk: 53204) value: 5


### Advanced Example: Nested Graph Inputs and Outputs

Graph inputs and outputs can be **nested**, allowing you to organize related parameters and results more effectively.

In [2]:
from aiida_workgraph import WorkGraph

wg = WorkGraph(name="test_inputs_outputs")
# Define both flat and nested graph-level inputs
wg.inputs = {"x": 1}
# Add a task using both top-level and nested inputs
wg.add_task(add, name="add1", x=wg.inputs.x, y=2)
wg.add_task(add, name="add2", x=wg.inputs.x, y=3)
# Define graph-level outputs by collecting results from the task
wg.outputs.results = {}
wg.outputs.results.sum1 = wg.tasks.add1.outputs.result
wg.outputs.results.sum2 = wg.tasks.add2.outputs.result
wg.run()
print("result: ", wg.outputs.results)

05/07/2025 01:02:55 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngine|continue_workgraph]: tasks ready to run: add1,add2
05/07/2025 01:02:55 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngine|update_task_state]: Task: add1, type: PyFunction, finished.
05/07/2025 01:02:56 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngine|continue_workgraph]: tasks ready to run: add2
05/07/2025 01:02:56 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngine|update_task_state]: Task: add2, type: PyFunction, finished.
05/07/2025 01:02:56 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
05/07/2025 01:02:56 PM <1220196> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [127391|WorkGraphEngi

result:  TaskSocketNamespace(name='results', sockets=['sum1', 'sum2'])


We can check the outputs of the WorkGraph process, and will fine the `sum1` and `sum2` are grouped under the `results` output:

In [3]:
!verdi process show {wg.pk}

Property     Value
-----------  ------------------------------------
type         WorkGraph<test_inputs_outputs>
state        Finished [0]
pk           127391
uuid         e8d980e8-fd52-492a-89e8-5a81d06ece18
label        test_inputs_outputs
description
ctime        2025-05-07 11:02:55.240178+00:00
mtime        2025-05-07 11:02:56.573200+00:00

Inputs                             PK      Type
---------------------------------  ------  ------
workgraph_data
    tasks
        add1
            inputs
                sockets
                    y
                        property
                            value  127389  Int
        add2
            inputs
                sockets
                    y
                        property
                            value  127390  Int

Outputs    PK      Type
---------  ------  ------
results
    sum1   127396  Int
    sum2   127401  Int

Called        PK  Type
--------  ------  ------
add1      127395  add
add2      127400  add

Log messages
--


### Summary

* Use `wg.inputs.<name>` to define reusable graph-level inputs.
* Use `wg.outputs.<name>` to expose relevant outputs from tasks.
* Graph inputs and outputs can be flat or nested for better organization.
* This approach encourages encapsulation and simplifies integration with other workflows or user interfaces.